<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_1DCNN_SCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving SCD.csv to SCD.csv


In [2]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['SCD.csv']),header=None)
df2.head()

,0,1,2,3,4,5,6,7,8,9
0,100.0,3000.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0
1,100.0,4300.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0
2,185.5,4823.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0
3,185.5,5008.5,1.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0
4,500.0,26000.0,0.0,0.0,1.0,1.0,800.0,677.2,6.0,1.0


In [0]:
features =df2.drop(df2.columns[9],axis=1)
labels=df2[df2.columns[9]]

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[-1.42200799 -0.67856862 -0.13742882  1.84422371  1.83180896  3.74165739
 -0.26978337 -0.25664688 -0.25310881]
(2460, 9)
(615, 9)
(2460,)
(615,)


In [5]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [6]:
X_train=X_train.reshape(2460,1,9)
X_test=X_test.reshape(615,1,9)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1,9)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [8]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m,precision_m, recall_m])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 2460 samples, validate on 615 samples
Epoch 1/10





2460/2460 [==============================] - 1s 423us/step - loss: 0.2744 - acc: 0.9232 - f1_m: 0.7679 - precision_m: 0.8258 - recall_m: 0.7778 - val_loss: 0.0769 - val_acc: 0.9724 - val_f1_m: 0.9154 - val_precision_m: 0.8956 - val_recall_m: 0.9496
Epoch 2/10
2460/2460 [==============================] - 0s 77us/step - loss: 0.0686 - acc: 0.9760 - f1_m: 0.9095 - precision_m: 0.9370 - recall_m: 0.9025 - val_loss: 0.0577 - val_acc: 0.9789 - val_f1_m: 0.9311 - val_precision_m: 0.9414 - val_recall_m: 0.9305
Epoch 3/10
2460/2460 [==============================] - 0s 75us/step - loss: 0.0535 - acc: 0.9789 - f1_m: 0.9190 - precision_m: 0.9422 - recall_m: 0.9159 - val_loss: 0.0587 - val_acc: 0.9821 - val_f1_m: 0.9410 - val_precision_m: 0.9417 - val_recall_m: 0.9474
Epoch 4/10
2460/2460 [==============================] - 0s 63us/step - loss: 0.0514 - acc: 0.9825 - f1_m: 0.9315 - precision_m: 0.9496 - recall_m: 0.9328 - val_loss: 0

In [10]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 5.57%
acc: 98.05%
f1: 92.49%
precision: 96.95%
recall: 89.41%


In [0]:
y_pred=model.predict_classes(X_test)

In [12]:
from sklearn import metrics
print("Accuracy: "+"{:.2%}".format(metrics.accuracy_score(y_test, y_pred)));
print("Precision: "+"{:.2%}".format(metrics.precision_score(y_test, y_pred)));
print("Recall: "+"{:.2%}".format(metrics.recall_score(y_test, y_pred)));
print("F1: "+"{:.2%}".format(metrics.f1_score(y_test,y_pred)));
print(metrics.confusion_matrix(y_test,y_pred));

Accuracy: 98.05%
Precision: 97.89%
Recall: 90.29%
F1: 93.94%
[[510   2]
 [ 10  93]]


In [12]:
for i in range(len(y_test)):
  if(y_test[i]==1):
    print(i)
    print(y_test[i])
    print(prediction[i])
    print("-----")

11
1.0
[0.99999994]
-----
12
1.0
[0.99999905]
-----
21
1.0
[0.9999999]
-----
22
1.0
[1.]
-----
23
1.0
[0.9999119]
-----
39
1.0
[0.99996924]
-----
41
1.0
[0.84004664]
-----
44
1.0
[0.99999917]
-----
48
1.0
[0.9999839]
-----
51
1.0
[0.9989784]
-----
53
1.0
[0.95010746]
-----
56
1.0
[0.99999976]
-----
60
1.0
[0.99997044]
-----
68
1.0
[0.92600733]
-----
69
1.0
[0.65357924]
-----
76
1.0
[0.9999368]
-----
87
1.0
[0.9999486]
-----
88
1.0
[0.99987894]
-----
90
1.0
[0.9999929]
-----
96
1.0
[0.04588977]
-----
102
1.0
[0.9985573]
-----
103
1.0
[0.99455756]
-----
108
1.0
[0.99937385]
-----
118
1.0
[0.9999614]
-----
124
1.0
[0.99977505]
-----
131
1.0
[0.93436885]
-----
133
1.0
[0.51255053]
-----
134
1.0
[0.99999964]
-----
139
1.0
[0.9999796]
-----
141
1.0
[0.9999811]
-----
142
1.0
[0.5994539]
-----
161
1.0
[0.99990964]
-----
164
1.0
[0.9999999]
-----
184
1.0
[0.70710516]
-----
188
1.0
[0.9535006]
-----
193
1.0
[0.9758302]
-----
194
1.0
[0.9959053]
-----
202
1.0
[0.9998948]
-----
206
1.0
[0.9843086]